In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, DecoderSimpleRNN, EncoderSimpleRNN, \
    Attention, DecoderAttnRNN, DecoderRNNV2, EncoderDecoderAttention, EncoderSimpleRNN_Test, DecoderAttnRNN_Test, \
    DecoderRNN_Test
from train_eval import count_parameters, summarize_results, \
    plot_single_learning_curve, load_experiment_log
from train_eval import train_and_eval, train_and_eval_attn 
import importlib
import pickle as pkl 
import torch

In [2]:
# model identification
MODEL_NAME = 'zh-seq2seq-rnn-attention'
SRC_LANG = 'zh'
TARG_LANG = 'en'

# data processing params  
SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 #30000
TARG_VOCAB_SIZE = 30000 #30000

# model architecture params 
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 #2 
ENC_HIDDEN_DIM = 256 #512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM #2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 # to actually implement
DEC_DROPOUT = 0.2 # to actually implement

# training params  
BATCH_SIZE = 32 #32
NUM_EPOCHS = 200
LR = 0.0005 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = True

In [3]:
# store as dict to save to results later 
params = {'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 'rnn_cell_type': RNN_CELL_TYPE, 
          'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 
          'src_vocab_size': SRC_VOCAB_SIZE, 'targ_vocab_size': TARG_VOCAB_SIZE, 
          'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 'dec_hidden_dim': DEC_HIDDEN_DIM,
          'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 'clip_grad_max_norm': CLIP_GRAD_MAX_NORM,
          'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 
          'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 'learning_rate': LR, 'optimizer': OPTIMIZER, 
          'lazy_train': LAZY_TRAIN} 

In [4]:
#vocab_test = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)

In [5]:
#vocab['zh']['id2token'][987]

In [6]:
#vocab['zh']['token2id']['森林']

In [7]:
#vocab['en']['token2id']['activity']

In [8]:
#vocab['en']['id2token'][987]

In [9]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [10]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, vocab)
data_minibatch = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=1000)

In [11]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab-fake.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [12]:
# vocab_filename = "{}-{}-vocab-fake.p".format(SRC_LANG, TARG_LANG)
# vocab = pkl.load(open(vocab_filename, "rb"))
# data = process_data(SRC_LANG, TARG_LANG, vocab)
# limited_data = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 

In [13]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [14]:
# define model 

# encoder = EncoderRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
#                      pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
# encoder = EncoderSimpleRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
#                            pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
# encoder = EncoderSimpleRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
#                            pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
encoder = EncoderSimpleRNN_Test(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
                                enc_dropout=ENC_DROPOUT, pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

# decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
#                       targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                       pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderRNNV2(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                        targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                        pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderSimpleRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                            targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                            pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderAttnRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                          targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
#                          targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                          pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderAttnRNN_Test(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                          targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
#                          targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
#                          pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
decoder = DecoderRNN_Test(
    rnn_cell_type=RNN_CELL_TYPE, attn=True, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
    num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
    targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
    pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))

model = EncoderDecoderAttention(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [15]:
model, results = train_and_eval_attn(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=True, save_checkpoint=True, save_to_log=True, 
    lazy_eval=False, inspect_samples=1)

Epoch: 0.00, Train Loss: 10.12, Val Loss: 10.24, Train BLEU: 0.31, Val BLEU: 0.20
Sampling from training predictions...
Source: 泰坦 泰坦尼克 泰坦尼克号 坦尼 尼克 号 是 拿 了 不少
Reference: the truth of the matter is that the titanic
Model: <SOS> it the the the the the the the the
Attention Weights: tensor([[0.0976, 0.0948, 0.0933, 0.0972, 0.0977, 0.1007, 0.1033, 0.1045, 0.1055,
         0.1054],
        [0.0976, 0.0949, 0.0934, 0.0973, 0.0977, 0.1007, 0.1032, 0.1044, 0.1054,
         0.1054],
        [0.0977, 0.0949, 0.0934, 0.0973, 0.0977, 0.1007, 0.1032, 0.1044, 0.1054,
         0.1053],
        [0.0977, 0.0949, 0.0934, 0.0973, 0.0977, 0.1007, 0.1032, 0.1044, 0.1054,
         0.1053],
        [0.0976, 0.0949, 0.0934, 0.0973, 0.0977, 0.1007, 0.1032, 0.1044, 0.1054,
         0.1053],
        [0.0976, 0.0949, 0.0934, 0.0973, 0.0977, 0.1007, 0.1032, 0.1044, 0.1054,
         0.1053],
        [0.0976, 0.0949, 0.0934, 0.0973, 0.0977, 0.1007, 0.1032, 0.1044, 0.1054,
         0.1053],
        [0.0976, 0.0949, 0

Epoch: 4.00, Train Loss: 8.39, Val Loss: 9.58, Train BLEU: 0.28, Val BLEU: 0.19
Sampling from training predictions...
Source: 其实 实地 地球 上 最长 的 山脉 都 在 海洋
Reference: and in the oceans , there are the longest
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0612, 0.0898, 0.1079, 0.1261, 0.1271, 0.1294, 0.1204, 0.1093, 0.0829,
         0.0459],
        [0.0595, 0.0889, 0.1079, 0.1270, 0.1282, 0.1307, 0.1214, 0.1097, 0.0822,
         0.0445],
        [0.0590, 0.0887, 0.1080, 0.1274, 0.1287, 0.1312, 0.1217, 0.1098, 0.0818,
         0.0439],
        [0.0588, 0.0887, 0.1080, 0.1275, 0.1288, 0.1313, 0.1217, 0.1097, 0.0817,
         0.0437],
        [0.0589, 0.0887, 0.1081, 0.1275, 0.1288, 0.1313, 0.1217, 0.1097, 0.0816,
         0.0437],
        [0.0590, 0.0888, 0.1081, 0.1275, 0.1288, 0.1312, 0.1216, 0.1096, 0.0816,
         0.0438],
        [0.0591, 0.0888, 0.1081, 0.1274, 0.1287, 0.1312, 0.1216, 0.1096, 0.0816,
         0.0438],
        [0.0591, 0.0889, 0.1081, 0

Epoch: 8.00, Train Loss: 6.46, Val Loss: 8.90, Train BLEU: 0.28, Val BLEU: 0.19
Sampling from training predictions...
Source: 大多 大多数 多数 地震 和 火山 喷发 也 都 发生
Reference: most of the earthquakes and volcanoes are in the
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0243, 0.0450, 0.1048, 0.1369, 0.1536, 0.1503, 0.1453, 0.1303, 0.0855,
         0.0241],
        [0.0200, 0.0398, 0.1022, 0.1387, 0.1586, 0.1553, 0.1495, 0.1323, 0.0827,
         0.0208],
        [0.0196, 0.0393, 0.1019, 0.1390, 0.1593, 0.1560, 0.1500, 0.1323, 0.0821,
         0.0204],
        [0.0195, 0.0393, 0.1018, 0.1390, 0.1594, 0.1561, 0.1501, 0.1323, 0.0821,
         0.0203],
        [0.0195, 0.0392, 0.1017, 0.1390, 0.1595, 0.1562, 0.1501, 0.1323, 0.0821,
         0.0203],
        [0.0195, 0.0392, 0.1017, 0.1390, 0.1595, 0.1562, 0.1501, 0.1323, 0.0821,
         0.0204],
        [0.0195, 0.0393, 0.1017, 0.1390, 0.1594, 0.1562, 0.1501, 0.1323, 0.0821,
         0.0204],
        [0.0195, 0.0393, 

Epoch: 12.00, Train Loss: 5.13, Val Loss: 8.64, Train BLEU: 0.28, Val BLEU: 0.19
Sampling from training predictions...
Source: 原来 它 是 海洋 洋中 最长 的 生物 <EOS> <PAD>
Reference: this turns out to be the longest creature in
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[4.1061e-02, 1.2376e-01, 1.5786e-01, 1.6129e-01, 1.3720e-01, 1.5914e-01,
         1.4181e-01, 7.1598e-02, 6.2811e-03, 3.5813e-10],
        [3.4871e-02, 1.2066e-01, 1.6063e-01, 1.6532e-01, 1.3904e-01, 1.6356e-01,
         1.4323e-01, 6.7415e-02, 5.2709e-03, 2.3881e-09],
        [3.4640e-02, 1.2031e-01, 1.6051e-01, 1.6536e-01, 1.3926e-01, 1.6376e-01,
         1.4350e-01, 6.7430e-02, 5.2320e-03, 2.7607e-09],
        [3.4593e-02, 1.2022e-01, 1.6038e-01, 1.6527e-01, 1.3935e-01, 1.6379e-01,
         1.4362e-01, 6.7544e-02, 5.2317e-03, 2.7052e-09],
        [3.4605e-02, 1.2021e-01, 1.6032e-01, 1.6521e-01, 1.3937e-01, 1.6377e-01,
         1.4365e-01, 6.7621e-02, 5.2455e-03, 2.6572e-09],
        [3.4602e-02, 

Epoch: 16.00, Train Loss: 4.48, Val Loss: 8.96, Train BLEU: 0.36, Val BLEU: 0.28
Sampling from training predictions...
Source: 我们 将 用 一些 影片 来讲 讲述 一些 深海 海里
Reference: and we &apos;re going to tell you some stories
Model: <SOS> it the the the the the , , ,
Attention Weights: tensor([[0.0360, 0.0929, 0.1135, 0.1218, 0.1264, 0.1285, 0.1283, 0.1235, 0.0996,
         0.0294],
        [0.0299, 0.0902, 0.1150, 0.1250, 0.1304, 0.1326, 0.1318, 0.1250, 0.0959,
         0.0244],
        [0.0300, 0.0901, 0.1148, 0.1249, 0.1302, 0.1325, 0.1317, 0.1250, 0.0961,
         0.0247],
        [0.0297, 0.0898, 0.1147, 0.1249, 0.1303, 0.1326, 0.1318, 0.1252, 0.0963,
         0.0248],
        [0.0295, 0.0896, 0.1146, 0.1249, 0.1303, 0.1327, 0.1319, 0.1253, 0.0964,
         0.0248],
        [0.0295, 0.0896, 0.1146, 0.1249, 0.1303, 0.1327, 0.1319, 0.1253, 0.0965,
         0.0248],
        [0.0295, 0.0896, 0.1146, 0.1249, 0.1303, 0.1327, 0.1319, 0.1253, 0.0965,
         0.0248],
        [0.0294, 0.0895, 0.1146, 

...

Epoch: 183.00, Train Loss: 0.89, Val Loss: 13.04, Train BLEU: 75.59, Val BLEU: 0.45
Sampling from training predictions...
Source: 泰坦 泰坦尼克 泰坦尼克号 坦尼 尼克 号 是 拿 了 不少
Reference: the truth of the matter is that the titanic
Model: <SOS> the truth of the matter is the the titanic
Attention Weights: tensor([[0.0158, 0.0088, 0.0071, 0.0145, 0.1057, 0.1215, 0.1796, 0.2042, 0.1852,
         0.1576],
        [0.0215, 0.0209, 0.0160, 0.0311, 0.1161, 0.1234, 0.1662, 0.1812, 0.1653,
         0.1580],
        [0.0478, 0.1139, 0.0752, 0.1112, 0.1118, 0.0969, 0.1090, 0.1118, 0.1059,
         0.1165],
        [0.0539, 0.2035, 0.1194, 0.1520, 0.0875, 0.0705, 0.0746, 0.0756, 0.0733,
         0.0898],
        [0.0568, 0.2762, 0.1528, 0.1676, 0.0668, 0.0524, 0.0534, 0.0535, 0.0525,
         0.0681],
        [0.0590, 0.3777, 0.1882, 0.1668, 0.0389, 0.0293, 0.0298, 0.0308, 0.0316,
         0.0478],
        [0.0604, 0.3939, 0.1925, 0.1632, 0.0350, 0.0261, 0.0266, 0.0278, 0.0289,
         0.0455],
        [0.0603,

Epoch: 187.00, Train Loss: 0.83, Val Loss: 13.09, Train BLEU: 78.30, Val BLEU: 0.39
Sampling from training predictions...
Source: 当 你 站 在 海滩 上 或是 当 你 看到
Reference: part of the problem , i think , is
Model: <SOS> part of the problem , , , , ,
Attention Weights: tensor([[0.0259, 0.0582, 0.0944, 0.1374, 0.2400, 0.1294, 0.1378, 0.0795, 0.0570,
         0.0403],
        [0.0301, 0.0574, 0.0894, 0.1290, 0.2462, 0.1194, 0.1413, 0.0762, 0.0571,
         0.0539],
        [0.0480, 0.0640, 0.0857, 0.1131, 0.2078, 0.1047, 0.1294, 0.0780, 0.0673,
         0.1020],
        [0.0566, 0.0669, 0.0840, 0.1065, 0.1924, 0.0988, 0.1236, 0.0780, 0.0695,
         0.1237],
        [0.0529, 0.0613, 0.0782, 0.1003, 0.1973, 0.0935, 0.1250, 0.0762, 0.0697,
         0.1455],
        [0.0504, 0.0573, 0.0742, 0.0956, 0.1988, 0.0909, 0.1266, 0.0757, 0.0701,
         0.1605],
        [0.0493, 0.0557, 0.0723, 0.0935, 0.2014, 0.0893, 0.1267, 0.0747, 0.0696,
         0.1676],
        [0.0481, 0.0543, 0.0707, 0.0921, 0.205

Epoch: 191.00, Train Loss: 0.78, Val Loss: 13.11, Train BLEU: 79.78, Val BLEU: 0.41
Sampling from training predictions...
Source: 还有 这些 摇晃 着 旋转 转着 的 触角 <EOS> <PAD>
Reference: it &apos;s got tentacles dangling , swirling around like
Model: <SOS> it &apos;s got tentacles dangling , swirling around like
Attention Weights: tensor([[7.7244e-02, 1.7028e-01, 2.0916e-01, 2.1447e-01, 2.8408e-01, 3.3307e-02,
         6.0463e-03, 4.2268e-03, 1.1896e-03, 2.0122e-10],
        [8.0110e-02, 1.4813e-01, 1.8634e-01, 1.9399e-01, 2.9973e-01, 7.3016e-02,
         8.9189e-03, 6.7474e-03, 3.0207e-03, 8.6300e-10],
        [7.5967e-02, 1.0273e-01, 1.2979e-01, 1.3293e-01, 2.3728e-01, 2.6536e-01,
         1.7389e-02, 1.6234e-02, 2.2313e-02, 1.1953e-07],
        [7.5220e-02, 8.8837e-02, 1.0797e-01, 1.0903e-01, 1.9213e-01, 3.4446e-01,
         2.3223e-02, 2.1880e-02, 3.7245e-02, 2.8354e-06],
        [6.5956e-02, 7.6643e-02, 9.5540e-02, 9.8010e-02, 1.8607e-01, 3.9342e-01,
         2.0536e-02, 2.0372e-02, 4.3440e-0

Epoch: 195.00, Train Loss: 0.72, Val Loss: 13.12, Train BLEU: 82.03, Val BLEU: 0.40
Sampling from training predictions...
Source: 我们 得用 非常 特殊 的 仪器 才能 能到 到达 那个
Reference: we have to have a very special technology to
Model: <SOS> we have to have a very special technology technology
Attention Weights: tensor([[0.0020, 0.0032, 0.1169, 0.1384, 0.1486, 0.1988, 0.3226, 0.0498, 0.0140,
         0.0059],
        [0.0046, 0.0154, 0.1189, 0.1278, 0.1314, 0.1785, 0.3006, 0.0908, 0.0213,
         0.0106],
        [0.0143, 0.0882, 0.1060, 0.0920, 0.0939, 0.1284, 0.2324, 0.1836, 0.0380,
         0.0232],
        [0.0198, 0.1296, 0.0981, 0.0810, 0.0833, 0.1126, 0.2076, 0.1986, 0.0418,
         0.0276],
        [0.0234, 0.1684, 0.0873, 0.0715, 0.0742, 0.1025, 0.1967, 0.2072, 0.0406,
         0.0283],
        [0.0255, 0.2084, 0.0764, 0.0619, 0.0646, 0.0926, 0.1900, 0.2159, 0.0375,
         0.0272],
        [0.0264, 0.2416, 0.0677, 0.0542, 0.0570, 0.0859, 0.1893, 0.2185, 0.0341,
         0.0253],
       

Epoch: 199.00, Train Loss: 0.67, Val Loss: 13.17, Train BLEU: 83.98, Val BLEU: 0.39
Sampling from training predictions...
Source: 和 我们 合作 的 人们 帮 我们 找到 了 新
Reference: people that have partnered with us have given us
Model: <SOS> people &apos;s have partnered with us us given us
Attention Weights: tensor([[0.0231, 0.0781, 0.1574, 0.1099, 0.1459, 0.1177, 0.1494, 0.1422, 0.0479,
         0.0286],
        [0.0272, 0.0748, 0.1486, 0.1039, 0.1505, 0.1089, 0.1563, 0.1524, 0.0465,
         0.0310],
        [0.0427, 0.0793, 0.1359, 0.1006, 0.1416, 0.1021, 0.1467, 0.1525, 0.0542,
         0.0443],
        [0.0530, 0.0826, 0.1287, 0.0986, 0.1338, 0.0997, 0.1387, 0.1513, 0.0610,
         0.0526],
        [0.0541, 0.0811, 0.1267, 0.0969, 0.1325, 0.0984, 0.1383, 0.1540, 0.0626,
         0.0556],
        [0.0488, 0.0732, 0.1231, 0.0894, 0.1355, 0.0916, 0.1451, 0.1753, 0.0611,
         0.0569],
        [0.0488, 0.0711, 0.1192, 0.0868, 0.1348, 0.0892, 0.1459, 0.1828, 0.0626,
         0.0587],
        [0

In [16]:
summarize_results(load_experiment_log())[['dt_created', 'num_epochs', 'learning_rate', 'clip_grad_max_norm', 'val_loss']].head()

,dt_created,num_epochs,learning_rate,clip_grad_max_norm,val_loss
76,2018-12-05 19:40:19,200,0.0005,1.0,8.638109
75,2018-12-05 12:50:28,200,0.0005,1.0,8.559556
74,2018-12-05 12:01:54,200,0.0005,1.0,8.550900
73,2018-12-05 00:57:30,200,0.0005,1.0,8.644947
72,2018-12-05 00:36:58,200,0.0005,1.0,8.638907


In [ ]:
plot_single_learning_curve(results)

In [ ]:
# Epoch: 199.00, Train Loss: 0.32, Val Loss: 13.19, Train BLEU: 98.94, Val BLEU: 0.27
plot_single_learning_curve(results)

In [ ]:
# with attention energies = v_broadcast.bmm(torch.tanh(self.attn(concat)).transpose(1, 2)) # switched order  
# Epoch: 199.00, Train Loss: 0.63, Val Loss: 12.82, Train BLEU: 92.05, Val BLEU: 0.38
plot_single_learning_curve(results)

In [ ]:
for i, token in enumerate(vocab[SRC_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
for i, token in enumerate(vocab[TARG_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
import torch
x = torch.arange(0, 3*5*10).view(3, 5, 10)
print(x)
y = x[1:, :, :]
print(y)
z = y.view(-1, 10)
print(z)

In [ ]:
t = torch.arange(0, 2*5).view(5, 2)
print(t)
u = t.contiguous().view(-1)
print(u)
v = t.permute(1, 0)
print(v)
w = v.contiguous().view(-1)
print(w)

In [ ]:
a = torch.arange(0, 2*1*300)
print(a)
b = a.view(-1, 1, 300)
print(b.size())

In [ ]:
for i, (src_idxs, targ_idxs, src_lens, targ_lens) in enumerate(full_loaders['train']):
#     print(i)
#     print(src_idxs.size())
#     print(src_idxs)
#     print(src_lens)
#     print(targ_idxs.size())
#     print(targ_idxs)
#     print(targ_lens)
    id2token = vocab[SRC_LANG]['id2token']
    test_tensor = src_idxs
    list_of_lists = test_tensor.numpy().astype(int).tolist()
    to_token = lambda l: ' '.join([id2token[idx] for idx in l])
    list_of_lists_tokens = [to_token(l) for l in list_of_lists] 
    break 